In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def read_data(filename):
    df = pd.read_csv(filename, encoding='utf-8', sep=',')
    return df


def calc_param(df):        
    tempc = df['TempC']
    tempk = tempc + 273.15
    pressure = df['Pbar']
    buffer_mineral = df['fO2']
    delta_redox = df['fO2delta']
    return tempc, tempk, pressure, buffer_mineral, delta_redox


def calc_wt(df):
    wt_liq_si = df['SiO2']
    wt_liq_ti = df['TiO2']
    wt_liq_al = df['Al2O3']
    wt_liq_fe = df['FeO']
    wt_liq_mn = df['MnO']
    wt_liq_mg = df['MgO']
    wt_liq_ca = df['CaO']
    wt_liq_na = df['Na2O']
    wt_liq_k  = df['K2O']
    wt_total = wt_liq_si + wt_liq_ti + wt_liq_al + wt_liq_fe + wt_liq_mn + wt_liq_mg + wt_liq_ca + wt_liq_na + wt_liq_k
    return wt_total, wt_liq_si, wt_liq_ti, wt_liq_al, wt_liq_fe, wt_liq_mn, wt_liq_mg, wt_liq_ca, wt_liq_na, wt_liq_k

def calc_molar(wt_liq_si, wt_liq_ti, wt_liq_al, wt_liq_fe, wt_liq_mn, wt_liq_mg, wt_liq_ca, wt_liq_na, wt_liq_k):   
    # calculate moles
    mol_liq_si = wt_liq_si / 60.0843
    mol_liq_ti = wt_liq_ti / 79.866
    mol_liq_al = wt_liq_al / 101.9613
    mol_liq_fe = wt_liq_fe / 71.8464
    mol_liq_mn = wt_liq_mn / 70.9374
    mol_liq_mg = wt_liq_mg / 40.304
    mol_liq_ca = wt_liq_ca / 56.077
    mol_liq_na = wt_liq_na / 61.9789
    mol_liq_k  = wt_liq_k  / 94.195
    mol_liq_sum = mol_liq_si + mol_liq_ti + mol_liq_al + mol_liq_fe + mol_liq_mn \
    + mol_liq_mg + mol_liq_ca + mol_liq_na + mol_liq_k
    
    # Calculate mole fractions
    x_liq_si = mol_liq_si / mol_liq_sum
    x_liq_al = mol_liq_al / mol_liq_sum
    x_liq_fe = mol_liq_fe / mol_liq_sum
    x_liq_mg = mol_liq_mg / mol_liq_sum
    x_liq_ca = mol_liq_ca / mol_liq_sum
    x_liq_na = mol_liq_na / mol_liq_sum
    x_liq_k  = mol_liq_k  / mol_liq_sum
    # print(x_liq_si + x_liq_al + x_liq_fe + x_liq_mg + x_liq_ca + x_liq_na + x_liq_k)
    # print(x_liq_si, x_liq_al, x_liq_fe, x_liq_mg, x_liq_ca, x_liq_na, x_liq_k)
    return x_liq_si, x_liq_al, x_liq_fe, x_liq_mg, x_liq_ca, x_liq_na, x_liq_k
    

def calc_cation(wt_liq_si, wt_liq_ti, wt_liq_al, wt_liq_fe, wt_liq_mn, wt_liq_mg, wt_liq_ca, wt_liq_na, wt_liq_k):
    # calculate moles
    mol_liq_si = wt_liq_si / 60.0843
    mol_liq_ti = wt_liq_ti / 79.866
    mol_liq_al = 2 * wt_liq_al / 101.9613
    mol_liq_fe = wt_liq_fe / 71.8464
    mol_liq_mn = wt_liq_mn / 70.9374
    mol_liq_mg = wt_liq_mg / 40.304
    mol_liq_ca = wt_liq_ca / 56.077
    mol_liq_na = 2 * wt_liq_na / 61.9789
    mol_liq_k  = 2 * wt_liq_k  / 94.195
    mol_liq_sum = mol_liq_si + mol_liq_ti + mol_liq_al + mol_liq_fe + mol_liq_mn \
    + mol_liq_mg + mol_liq_ca + mol_liq_na + mol_liq_k
    
    # Calculate mole fractions
    cation_liq_si = mol_liq_si / mol_liq_sum
    cation_liq_al = mol_liq_al / mol_liq_sum
    cation_liq_fe = mol_liq_fe / mol_liq_sum
    cation_liq_mn = mol_liq_mn / mol_liq_sum
    cation_liq_mg = mol_liq_mg / mol_liq_sum
    cation_liq_ca = mol_liq_ca / mol_liq_sum
    cation_liq_na = mol_liq_na / mol_liq_sum
    cation_liq_k  = mol_liq_k  / mol_liq_sum
    # print(x_liq_si + x_liq_al + x_liq_fe + x_liq_mg + x_liq_ca + x_liq_na + x_liq_k)
    # print(x_liq_si, x_liq_al, x_liq_fe, x_liq_mg, x_liq_ca, x_liq_na, x_liq_k)
    return cation_liq_si, cation_liq_al, cation_liq_fe, cation_liq_mn, cation_liq_mg, cation_liq_ca, cation_liq_na, cation_liq_k

# The dataset was quoted from Kress and Carmichael (1991)
# Ratio: Fe2O3/FeO
def calcredox(model, tempk, fo2, x_liq_si, x_liq_al, x_liq_fe, x_liq_mg, x_liq_ca, x_liq_na, x_liq_k):
    if model == 'Sack':
        ln_ratio = ( 0.218130 * np.log(fo2) ) + ( 13184.7 / tempk ) - 4.49933 \
        + ( ( - 2.15036 * x_liq_si ) + ( - 8.35163 * x_liq_al ) + ( - 4.49508 * x_liq_fe ) + ( - 5.43639 * x_liq_mg ) \
           + ( 0.073113 * x_liq_ca ) + ( 3.54148 * x_liq_na ) + ( 4.18688* x_liq_k ) )
    elif model == 'Kress':
        ln_ratio = ( 0.218130 * np.log(fo2) ) + ( 13184.7 / tempk ) - 4.49933 \
        + ( ( - 2.15036 * x_liq_si ) + ( - 8.35163 * x_liq_al ) + ( - 4.49508 * x_liq_fe ) + ( - 5.43639 * x_liq_mg ) \
        + ( 0.073113 * x_liq_ca ) + ( 3.54148 * x_liq_na ) + ( 4.18688* x_liq_k ) )
    else:
        print('Error!')
    coef = 1 / ( 1 + 2 * np.exp(ln_ratio) )
    return coef


def calc_fo2(buffer_mineral, delta_redox, tempk, pressure):
    # log10(fo2) = A/T[K] + B + C(P[bar]-1)/T[K]
    # QFM: Huebner (1971) and NNO: Huebner and Sato (1970)
    fo2 = []
    for i in range(len(buffer_mineral)):
        if buffer_mineral[i] == 'QFM':
            calc_fo2 = 10 ** ( ( - 25096.3 / tempk[i] ) + 8.735 + ( 0.11 * ( pressure[i] - 1 ) / tempk[i] ) + delta_redox[i] )
            fo2.append(calc_fo2)
        elif buffer_mineral[i] == 'NNO':
            calc_fo2 = 10 ** ( ( - 24930 / tempk[i] ) + 9.36 + ( 0.046 * ( pressure[i] - 1 ) / tempk[i] ) + delta_redox[i] )
            fo2.append(calc_fo2)
        else:
            print('Error!')
    return fo2


def calc_kd_femg(cation_liq_fe, cation_liq_mn, cation_liq_mg):
    # Beattie et al. (1991)
    d_mg = ( 0.666 - ( - 0.049 * cation_liq_mn + 0.027 * cation_liq_fe ) ) / \
    ( cation_liq_mg + 0.259 * cation_liq_mn + 0.299 * cation_liq_fe )
    kd_femg = 0.279 + 0.031 / d_mg
    # print(d_mg, kd_femg)
    return kd_femg


def main():
    filename = 'input.csv'
    # select Fe3+/Fe2+ model by Sack et al. (1980) or Kress and Carmichael (1991) 
    model = 'Kress'
    # read data set
    df = read_data(filename)    
    tempc, tempk, pressure, buffer_mineral, delta_redox = calc_param(df)
    wt_total, wt_liq_si, wt_liq_ti, wt_liq_al, wt_liq_fe, wt_liq_mn, wt_liq_mg, wt_liq_ca, wt_liq_na, wt_liq_k = calc_wt(df)
    # calc liquid molar fractions
    x_liq_si, x_liq_al, x_liq_fe, x_liq_mg, x_liq_ca, x_liq_na, x_liq_k \
    = calc_molar(wt_liq_si, wt_liq_ti, wt_liq_al, wt_liq_fe, wt_liq_mn, wt_liq_mg, wt_liq_ca, wt_liq_na, wt_liq_k)    
    # calc oxide fugacity
    fo2 = calc_fo2(buffer_mineral, delta_redox, tempk, pressure)
    # print(fo2)
    coef = calcredox(model, tempk, fo2, x_liq_si, x_liq_al, x_liq_fe, x_liq_mg, x_liq_ca, x_liq_na, x_liq_k)
    
    # ln_ratio = calcredox_kress(tempk, fo2, x_liq_al, x_liq_fe, x_liq_ca, x_liq_na, x_liq_k)
    # ratio, coef = coef_conv(ln_ratio)
    #print('Ratio: '+str(ratio), 'Coefficient: '+str(coef))
    
    cation_liq_si, cation_liq_al, cation_liq_fe, cation_liq_mn, cation_liq_mg, cation_liq_ca, cation_liq_na, cation_liq_k \
    = calc_cation(wt_liq_si, wt_liq_ti, wt_liq_al, wt_liq_fe, wt_liq_mn, wt_liq_mg, wt_liq_ca, wt_liq_na, wt_liq_k)
    # calc partition coefficient Fe-Mg in olivine
    kd_femg = calc_kd_femg(cation_liq_fe, cation_liq_mn, cation_liq_mg)
        
    # calc Fo
    fo = 1 / ( kd_femg * ( coef * x_liq_fe / x_liq_mg ) + 1 )
    mg_sisson1 = 1 / ( 0.23 * ( x_liq_fe / x_liq_mg ) + 1 )
    mg_sisson2 = 1 / ( 0.26 * ( 0.86 * x_liq_fe / x_liq_mg ) + 1 )
    mg_sisson3 = 1 / ( 0.26 * ( coef * x_liq_fe / x_liq_mg ) + 1 )
    mg_putirka1 = 1 / ( 0.28 * ( x_liq_fe / x_liq_mg ) + 1 )
    kd = np.exp( - 0.107 - 1719 / tempk )
    mg_putirka2 = 1 / ( kd * ( x_liq_fe  / x_liq_mg ) + 1 )
    mg_putirka3 = 1 / ( kd * ( coef * x_liq_fe  / x_liq_mg ) + 1 )
    print( 'SiO2wt'+str(wt_liq_si)+', Fo: '+str(fo) )
    #print( 'Cpx-Mg#(Sisson&Grove1): '+str(mg_sisson1), 'Cpx-Mg#(Sisson&Grove2): '+str(mg_sisson2), 'Cpx-Mg#(Sisson&Grove3): '+str(mg_sisson3) )
    #print( 'Cpx-Mg#(Putirka1): '+str(mg_putirka1), 'Cpx-Mg#(Putirka2): '+str(mg_putirka2), 'Cpx-Mg#(Putirka3): '+str(mg_putirka3) )
    
    fo.to_csv('result.csv')


if __name__ == "__main__":
    main()

SiO2wt0     48
1     49
2     50
3     51
4     52
      ..
64    66
65    67
66    68
67    69
68    70
Name: SiO2, Length: 69, dtype: int64, Fo: 0     0.808801
1     0.807903
2     0.806889
3     0.805747
4     0.804470
        ...   
64    0.741164
65    0.724264
66    0.700879
67    0.666510
68    0.611309
Length: 69, dtype: float64
